In [1]:
!nvidia-smi

Mon Feb 26 20:43:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# **Environment Setup and Dependencies Installation**

In [2]:
!pip install -q ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.8 MB/s eta 0:00:00
time: 272 µs (started: 2024-02-26 20:45:06 +00:00)


In [3]:
!pip install accelerate==0.21.0 transformers==4.31.0 tokenizers==0.13.3
!pip install bitsandbytes==0.40.0 einops==0.6.1
!pip install xformers==0.0.22.post7
!pip install langchain==0.1.4
!pip install faiss-gpu==1.7.1.post3
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.5 MB/s eta 0:00:00
     ━━

# **Model Initialization and Configuration**

In [4]:
from torch import cuda, bfloat16
import transformers

model_id = 'NousResearch/Llama-2-7b-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, you need an access token
hf_auth = 'add your access token here'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

# enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda122_nocublaslt.so


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

Model loaded on cuda:0
time: 3min 23s (started: 2024-02-26 20:47:04 +00:00)


# **Tokenizer Preparation**

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# Tokeninzer Preperation
tokenizer = AutoTokenizer.from_pretrained(
    model_id
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

time: 2.83 s (started: 2024-02-26 20:50:28 +00:00)


In [6]:
print(tokenizer)

LlamaTokenizerFast(name_or_path='NousResearch/Llama-2-7b-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
time: 553 µs (started: 2024-02-26 20:50:30 +00:00)


In [7]:
print(tokenizer.pad_token,tokenizer.eos_token)

<unk> </s>
time: 4 ms (started: 2024-02-26 20:50:30 +00:00)


In [8]:
tokenizer.pad_token = tokenizer.eos_token

time: 370 µs (started: 2024-02-26 20:50:30 +00:00)


In [9]:
tokenizer.truncation_side

'right'

time: 3.92 ms (started: 2024-02-26 20:50:30 +00:00)


In [10]:
tokenizer.truncation_side = 'left' # to prevent cutting off last generation

time: 343 µs (started: 2024-02-26 20:50:30 +00:00)


In [11]:
stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

[[1, 29871, 13, 29950, 7889, 29901], [1, 29871, 13, 28956, 13]]

time: 6.39 ms (started: 2024-02-26 20:50:30 +00:00)


In [12]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([    1, 29871,    13, 29950,  7889, 29901], device='cuda:0'),
 tensor([    1, 29871,    13, 28956,    13], device='cuda:0')]

time: 26.8 ms (started: 2024-02-26 20:50:30 +00:00)


# **Custom Stopping Criteria for Text Generation**

In [13]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

time: 729 µs (started: 2024-02-26 20:50:31 +00:00)


# **Generating Text with Custom Stopping Criteria**

In [23]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=2048,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

time: 967 µs (started: 2024-02-26 21:01:53 +00:00)


In [15]:
res = generate_text("How to deploy a model using Amazon Sagemaker?")
print(res[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


What to deploy a model using Amazon Sagemaker?
 nobody knows.

Comment: @MohamedAli I'm not sure what you mean by "deploying" the model, but if you want to use it in production, then you need to make sure that your model is well-tested and has been trained on data that is representative of the data you will be using it with. You also need to ensure that the model is able to handle the load (e.g., number of requests per second) that you expect it to receive.
time: 25.8 s (started: 2024-02-26 20:50:32 +00:00)


# **Web Document Loading and Processing**

In [26]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

# checking again that everything is working fine
llm(prompt="How to deploy a model using Amazon Sagemaker?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'\n sierp 2019.\n\n### 1. Prerequisites\n\n- Python 3.6+\n- Pipenv\n- Docker\n- AWS account\n- AWS CLI\n- AWS SageMaker Studio\n\n### 2. Create an environment\n\n```bash\npipenv install\n```\n\n### 3. Run the code\n\n```bash\npython main.py\n```\n\n### 4. Deploy the model\n\n```bash\nsagemaker deploy --image sagemaker-tensorflow:latest --model_name_override=mnist_tf_keras \\\n--role arn:aws:iam::<your-account>:role/SageMakerExecutionRole \\\n--region <your-region> \\\n--output_path./output \\\n--input_path./data/mnist/train.csv \\\n--output_path./output/mnist_tf_keras \\\n--instance_type ml.c5.xlarge\n```\n'

time: 21 s (started: 2024-02-26 21:03:54 +00:00)


# **Web Document Loading and Processing**

In [17]:
from langchain.document_loaders import WebBaseLoader

web_links = ['https://huggingface.co/docs/api-inference',
        'https://huggingface.co/docs/inference-endpoints',
        'https://huggingface.co/docs/optimum-neuron',  # Listed twice in your request, included once
        'https://huggingface.co/docs/sagemaker']

loader = WebBaseLoader(web_links)
documents = loader.load()

time: 1.91 s (started: 2024-02-26 20:51:55 +00:00)


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

time: 9.85 ms (started: 2024-02-26 20:51:57 +00:00)


# **Embeddings and Vector Store Setup**

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectorstore = FAISS.from_documents(all_splits, embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

time: 7.33 s (started: 2024-02-26 20:51:57 +00:00)


# **Conversational Retrieval Chain Implementation**

In [20]:
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

time: 1.35 s (started: 2024-02-26 20:52:04 +00:00)


In [21]:
chat_history = []

query = "How to deploy a model using Amazon Sagemaker?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




Amazon SageMaker is a fully managed service that makes it easy to build, train, and deploy machine learning models. It provides a wide range of capabilities, including prebuilt algorithms, tools for building custom models, and services for deploying those models into production. You can use SageMaker to build and train machine learning models, such as image classification or text analysis, and then deploy them to applications running on Amazon EC2 or other cloud providers.

To get started with SageMaker, sign in to the AWS Management Console and create a new SageMaker account. Once you’ve created your account, you can start building models by uploading data files or creating them directly within SageMaker Studio. After you’ve trained your model, you can deploy it to an application running on EC2 or another cloud provider.

SageMaker is designed to be easy to use, even for beginners. There are no complex configurations required; simply select the type of algorithm you want to use and 

In [27]:
chat_history = []

query = "Lay down the step-by-step procedure of training and deploying Llama2 on Amazon Sagemaker"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




The following steps will help you train and deploy a Llama2 model on Amazon Sagemaker.

1. Create a SageMaker Notebook Instance

You need to create a SageMaker notebook instance before you start training your model. You can do this by going to the SageMaker console and selecting "Create Notebook Instance". Once you have created the notebook instance, you will need to select the appropriate compute type (e.g., CPU or GPU) based on the size of your dataset and the complexity of your model.

2. Prepare Your Dataset

Once you have selected the appropriate compute type, you will need to prepare your dataset for training. This involves splitting your dataset into training and validation sets, as well as preprocessing the data so that it is ready for use by your model.

3. Train Your Model

Once you have prepared your dataset, you can begin training your model. To do this, you will need to specify the hyperparameters for your model, such as the number of epochs and the learning rate. You ca

In [28]:
chat_history = []

query = "What are Inference Endpoints? Why do we need them? Tell me about the pricing as well!"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Inference Endpoints allow you to run your own machine learning model locally without having to manage any infrastructure yourself. You can use our hosted Inference API to run your model on our servers. This allows you to focus on building your model instead of worrying about managing the underlying infrastructure.

The pricing for Inference Endpoints is based on the number of requests made to the endpoint. The more requests you make, the higher the cost will be. However, there are also discounts available if you sign up for a monthly plan.

Inference Endpoints are a great way to get started with machine learning without having to invest in expensive hardware or software. They provide a simple and easy way to start experimenting with different models and algorithms.

































































































































































































































In [29]:
chat_history = []

query = "How are Inference Endpoints different from API Inference?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Inference Endpoints are dedicated to serving requests for a single model. They can be used to serve requests for multiple models, but each endpoint will only serve requests for one model.




























































































































































































































































































































































































































































































time: 50.6 s (started: 2024-02-26 21:07:55 +00:00)
